In [ ]:
pip install fuzzywuzzy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import json
from fuzzywuzzy import process

# Load the names from the Excel file
excel_file_path = '/content/drive/MyDrive/Narrations/nbfc names.xlsx'
df_names = pd.read_excel(excel_file_path, engine='openpyxl')

# Load transactions from the JSON file
json_file_path = '/content/drive/MyDrive/Narrations/all_narrations (2).json'
with open(json_file_path, 'r') as f:
    transactions_data = json.load(f)

# Take only the first 10000000 transactions
transactions_data = transactions_data[:10000000]

stopwords = ['private', 'limited', 'pvt', 'ltd', 'ltd.',]

# Remove stopwords from names
def remove_stopwords(name):
    for sw in stopwords:
        name = str(name).lower()  # Ensure name is a string
        name = name.replace(sw, '').strip()
    return name

df_names['Name'] = df_names['Name'].apply(remove_stopwords)

# Create an empty list to store results
results = []

# Function to find transactions containing similar names
def find_transactions(name):
    transactions_found = []  # Using a set to store unique transactions  #complexity increasing
    for transaction in transactions_data:
        match = process.extractOne(name.upper(), transaction.upper())
        if match is not None:
            ratio = match[1]  # Get the similarity ratio
            if ratio >= 30:  # Threshold of 30% match
                transactions_found.append(transaction)
        return transactions_found

# Search for each name in transactions and store results
for name in df_names['Name'].head(5):  # Taking the first 100 names
    transactions_found = find_transactions(name)
    for transaction in transactions_found:
        transactions_list = transaction.split(',')  # Split transactions by ","
        for t in transactions_list:
            results.append({'Name': name, 'Narration': name, 'Transaction': t.strip()})

# Create DataFrame from results
result_df = pd.DataFrame(results)

# Write unique results back to the Excel file
excel_file_path = '/content/drive/MyDrive/Narrations/nbfc_names_output.xlsx'
excel_engine = 'openpyxl'

try:
    with pd.ExcelWriter(excel_file_path, engine=excel_engine) as writer:
        result_df.drop_duplicates().to_excel(writer, index=False)  # Drop duplicates before writing to Excel
    print("Data has been successfully written back to the Excel file.")
except Exception as e:
    print("An error occurred while writing to the Excel file:", e)


Data has been successfully written back to the Excel file.


In [ ]:
import pandas as pd
import json
from fuzzywuzzy import process

# Load the names from the Excel file
excel_file_path = '/content/drive/MyDrive/Narrations/nbfc names.xlsx'
df_names = pd.read_excel(excel_file_path, engine='openpyxl')

# Load transactions from the JSON file
json_file_path = '/content/drive/MyDrive/Narrations/all_narrations (2).json'
with open(json_file_path, 'r') as f:
    transactions_data = json.load(f)

# Take only the first 10000000 transactions
transactions_data = transactions_data[:10000000]

# Define stopwords to remove from names
stopwords = ['private', 'limited', 'pvt', 'ltd', 'ltd.',]

# Remove stopwords from names
def remove_stopwords(name):
    for sw in stopwords:
        name = str(name).lower()  # Ensure name is a string
        name = name.replace(sw, '').strip()
    return name

df_names['Name'] = df_names['Name'].apply(remove_stopwords)
df_names['Name']

0                                         bajaj finance .
1       shriram finance  [formerly: shriram transport ...
2                   tata sons   (formerly tata sons ) cic
3          cholamandalam investment and finance company *
4                                             l&t finance
                              ...                        
9351                                           ch finance
9352                                   blackbuck finserve
9353                                        edgro finance
9354                sanghamithra rural financial services
9355                    tally account aggregator services
Name: Name, Length: 9356, dtype: object